# Common instructions

- Aim to attempt all the mandatory questions (marked with a *) in the problem set. 


- Only attempt optional question after you have attempted all the mandatory questions. 


- More credit will be given if you have successfully attempted all the mandatory questions, even if you do not attempt a single optional question, as opposed to missing even one mandatory question while attempting all the optional questions.


- With the above caveat, attempt as many questions as possible within the time period. Partially attempted questions will get partial credit.


- Normally, you should work through the problem set in ascending order (Q1 -> Q4).


- Clean, labeled plots and clear data interpretation will boost your score. So too, will the use of functions, meaningful variable names, and readable code.


- You have a maximum of 2 days to work on the assignment. We will not consider assignments submitted after the deadline. You are free to search the internet, but are not to discuss with others in any way or form, in pain of immediate disqualification.


- Report the websites used to obtain help. Before the deadline, create a single .zip file with all your code submit it in the submission link provided to you in the email. DO NOT include data in your zip file. 


- You can use any programming language of your choice to solve all or part of the questions, preferably notebooks like Jupyter Notebook, Google Colab etc. We should be able to execute your program(s) to generate the required data and plots.


- In case you are unable to complete some parts, clearly indicate how would you go about the task ? What steps would you try etc.


# Input data

- The TSV file `SampleData.tsv` has the following columns

    - <b>Sample</b>: Sample IDs (S01, S02, S03...)
    - <b>Treatment</b>: Information on sample type 
    
        - <i>HF+</i>: Blood plasma samples collected from coronary disease patients post major surgery who had a heart failure within 3 years of surgery
        - <i>HF-</i>: Blood plasma samples collected from coronary disease patients post major surgery who recovered post surgery without heart failure
        - <i>HVOL</i>: Blood plasma samples collected from individuals without any discernable coronary disease
        
        
    
- The gzipped file `GSE208194_RawTPM.csv.gz` contains gene expression information for the sample mentioned in `SampleData.tsv` where the file structure looks like

ENSEMBL ID        |S01      |S02      |S03      |S04
:-----------------|:--------|:--------|:--------|:--------
ENSG00000000419.12|2.398878 |12.157726|1.40211  |7.667875
ENSG00000000938.13|3.324077 |13.971038|1.917631 |10.225812
ENSG00000001629.10|12.037059|1.453811 |12.596614|15.799738
ENSG00000001631.15|1.287932 |5.842868 |1.412257 |1.526812

Where each row is a feature/gene (n=4150) and each column is a sample where the features are measured.


# Feature filter*

- Remove genes/features with very low expression values (< 1) in 70% of samples (even if they are good features they cannot be used for diagnositics)
- Remove a member of feature pairs which are highly correlated
- Provide the statistics with respect to the removals

In [74]:
# Place your code here
import numpy as np
import pandas as pd

sample_path = 'SampleData.tsv'
dfSample = pd.read_table(sample_path,index_col=0)

In [75]:
genome_path = './GSE208194_RawTPM.csv'
df = pd.read_csv(genome_path,index_col=0)

In [76]:
df.columns

Index(['S01', 'S02', 'S03', 'S04', 'S05', 'S06', 'S07', 'S08', 'S09', 'S10',
       'S11', 'S12', 'S13', 'S14', 'S15', 'S16', 'S17', 'S18', 'S19', 'S20',
       'S21', 'S22', 'S23', 'S24', 'S26', 'S27', 'S28', 'S29', 'S30', 'S31',
       'S32', 'S33', 'S34', 'S35', 'S36', 'S37', 'S38', 'S39', 'S40', 'S41',
       'S42', 'S44', 'S45', 'S46', 'S47', 'S48', 'S49', 'S50', 'S51', 'S52',
       'S53', 'S54', 'S55', 'S56', 'S57', 'S58', 'S59', 'S60', 'S61', 'S62',
       'S63', 'S64', 'S65', 'S66', 'S67', 'S68', 'S69', 'S70', 'S71', 'S72',
       'S73', 'S74', 'S75', 'S77', 'S78', 'S79', 'S80', 'S81', 'S82', 'S83',
       'S84', 'S85', 'S86', 'S87', 'S88', 'S89', 'S90', 'S95', 'S96'],
      dtype='object')

In [77]:
for i in range(len(df)):
    curr_count=0
    for j in range(len(df.iloc[i])):
        if df.iloc[i,j]<1.0:
            curr_count+=1
    if curr_count/89 >0.7:
        print('True')
        df.drop(index=i)

In [78]:
print(df.shape) #No features removed as no feature has more than 70% values less than 1.0

(4150, 89)


In [79]:
df = df.transpose()

#Calculating Correlation Matrix
corr = df.corr().abs()

upper = corr.where(np.triu(np.ones(corr.shape),k=1).astype(bool))

drop_cols = [col for col in upper.columns if any(upper[col]>0.95)]
df.drop(drop_cols,axis=1,inplace=True)


In [80]:
df.shape # 164 features were dropped as they were highly related to another feature

(89, 3986)

1. We can see that we did not find any feature which has more than 70% of its value measured as less than 70%
2. Total of 164 pairs of features where found to be highly correlated. So these were dropped, leaving us with 3986 features.

# Feature elimination*

From the feature matrix `GSE208194_RawTPM.csv.gz`, split the samples into training and test and on the training set 
- using recursive feature elimination identify the optimal number of features out of the 4150, which can predict the status of a particular individual to be 
    - Having the likelihood of coronary disease which can lead to heart failure (HF+)
    - Having the likelihood of coronary disease which may not lead to heart failure (HF-)
    - Healthy indiviual (HVOL)
    
- Show the relative accuracy of the model as different number of features are used as a plot.

In [81]:
# Place your code here
#Attempting to find Proper number of features for a moel using SVC
from sklearn.svm import SVC 


#df = df.transpose() Uncomment for the first try

df_merged=df.merge(dfSample,how='inner',left_index=True,right_index=True)
df_merged_train = df[]

In [93]:
X = df_merged.drop(['treatment'],axis=1)
X_train = X[0:75]
X_test = X[75:]
y = df_merged['treatment']
y_train = y[0:75]
y_test = y[75:]

# visualizer = RFECV(SVC(kernel='linear',C=1))
# visualizer.fit(X,y)
# visualizer.show()



In [94]:
from sklearn.feature_selection import RFE

rfe = RFE(estimator=SVC(kernel='linear',C=1))

history = rfe.fit(X_train,y_train)

AttributeError: 'RFE' object has no attribute 'show'

In [96]:
print(X_train.columns[rfe.support_])

Index(['ENSG00000001629.10', 'ENSG00000001631.15', 'ENSG00000002549.12',
       'ENSG00000002586.20', 'ENSG00000003056.8', 'ENSG00000003402.20',
       'ENSG00000003436.16', 'ENSG00000003756.17', 'ENSG00000004059.11',
       'ENSG00000004700.16',
       ...
       'ENSG00000282508.2', 'ENSG00000283907.1', 'ENSG00000284770.2',
       'ENSG00000284809.1', 'ENSG00000284959.2', 'ENSG00000285756.2',
       'ENSG00000285774.2', 'ENSG00000285884.1', 'ENSG00000286171.1',
       'ENSG00000286522.1'],
      dtype='object', length=1993)


In [99]:
from sklearn.feature_selection  import RFECV

rfecv = RFECV(estimator=SVC(kernel='linear',C=1),min_features_to_select=50,step=5,n_jobs = -1,cv=5)
_ = rfecv.fit(X_train,y_train)


In [104]:
print(X_train.columns[rfecv.support_])

Index(['ENSG00000049245.13', 'ENSG00000065978.19', 'ENSG00000067225.18',
       'ENSG00000070756.16', 'ENSG00000085276.19', 'ENSG00000087086.15',
       'ENSG00000089335.21', 'ENSG00000092531.10', 'ENSG00000100461.18',
       'ENSG00000101162.4', 'ENSG00000102265.12', 'ENSG00000103187.8',
       'ENSG00000103363.14', 'ENSG00000103769.10', 'ENSG00000104267.10',
       'ENSG00000108654.15', 'ENSG00000108953.17', 'ENSG00000110880.10',
       'ENSG00000111644.8', 'ENSG00000112541.15', 'ENSG00000113558.18',
       'ENSG00000115652.14', 'ENSG00000115935.17', 'ENSG00000116251.11',
       'ENSG00000124491.16', 'ENSG00000128245.15', 'ENSG00000129515.20',
       'ENSG00000129562.11', 'ENSG00000130066.16', 'ENSG00000131171.13',
       'ENSG00000140367.12', 'ENSG00000142669.15', 'ENSG00000143546.10',
       'ENSG00000147065.17', 'ENSG00000150681.10', 'ENSG00000156411.9',
       'ENSG00000158869.11', 'ENSG00000160014.17', 'ENSG00000164924.18',
       'ENSG00000168300.14', 'ENSG00000168522.13', 'ENS

# Model building*

With N optimal features identified, used a feature selection method of choice to select the top N features and build a multi-class classifier with cross-validation and report its accuracy and any other metric deemed suitable.


In [ ]:
# Place your code here

# Model evaluation*

- Apply the model on the test set and report the accuracy and per class sensitivity i.e the total number of samples in a given class within the test set, and how many were predicted accurately.
- Also report as a binary matrix how many samples were correctly identified as HF+/HF- and HVOL

|    |      HF+/-      |  HVOL |
|----------|:-------------:|------:|
| HF+/- |  . | . |
| HVOL |    .   | . |


In [ ]:
# Place your code here

# Biological significance of features

- Take the top 5 features and search for their gene names in [Gene Cards](https://www.genecards.org/) with using the suffix for example:
  ENSG00000000419.12 -> ENSG00000000419	
  
- Take the gene names identified and search in [Pubmed](https://pubmed.ncbi.nlm.nih.gov/) as 
  "Gene name" AND "coronary disease"
  
- Report if anything of interest is found by reading the abstract of the top hits in pubmed.